In [13]:
!pip install dagshub mlflow --quiet

import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
import logging

logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)

warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")

print("Done!")

Done!


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [5]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import os
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
from prophet import Prophet

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

mlflow.set_experiment("Prophet_Training")

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

<Experiment: artifact_location='mlflow-artifacts:/cfb5b325e2b645bba453d65e11cf26d0', creation_time=1751725546879, experiment_id='9', last_update_time=1751725546879, lifecycle_stage='active', name='Prophet_Training', tags={}>

In [6]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["week"] = (
            X["Date"].dt.to_period("W")
            .rank(method="dense")
            .astype(int) - 1
        )
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)
        X = X.drop(columns=["Date"])
        return X

date_features = ["week", "sin_13", "cos_13", "sin_23", "cos_23"]

lag_features = []

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

added_features = date_features + lag_features

class ColumnTransformerWithNames(ColumnTransformer):
    def get_feature_names_out(self, input_features=None):
        return super().get_feature_names_out(input_features)

    def transform(self, X):
        X_transformed = super().transform(X)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

    def fit_transform(self, X, y=None):
        X_transformed = super().fit_transform(X, y)
        cols = self.get_feature_names_out()
        cols = [c.split("__", 1)[-1] for c in self.get_feature_names_out()]
        res = pd.DataFrame(X_transformed, columns=cols, index=X.index)
        return res

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

In [15]:
class ProphetRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, store_level='Store', dept_level='Dept'):
        self.store_level = store_level
        self.dept_level = dept_level

    def fit(self, X, y):
        if not isinstance(X.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")

        self.models_ = {}
        self.avgs_ = {}

        df = X.copy()
        df["y"] = y.values
        df = df.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)

        grouped = df.groupby([self.store_level, self.dept_level])

        for (store, dept), group_df in grouped:
            if dept == 1:
                print("store:", store)

            ts = group_df[["ds", "y"]].copy()

            try:
                model = Prophet(daily_seasonality=False, weekly_seasonality=True, yearly_seasonality=True)
                model.fit(ts)
                self.models_[(store, dept)] = model
            except Exception as e:
                print(f"Skipping ({store}, {dept}) due to error: {e}")
                self.skip_(ts["y"], store, dept)
                continue

        return self

    def skip_(self, ts, store, dept):
        if ts is None or len(ts) == 0:
            self.avgs_[(store, dept)] = 0.0
        else:
            self.avgs_[(store, dept)] = ts.mean()

    def predict(self, X):
        if not isinstance(X.index, pd.MultiIndex):
            raise ValueError("X must have a MultiIndex")

        df = X.copy()
        df = df.reset_index()
        df.rename(columns={"Date": "ds"}, inplace=True)

        preds = pd.Series(index=X.index, dtype=float)
        grouped = df.groupby([self.store_level, self.dept_level])

        for (store, dept), group_df in grouped:
            if dept == 1:
                print("store:", store)

            model = self.models_.get((store, dept))
            if model is None:
                preds.loc[group_df.set_index([self.store_level, self.dept_level, "ds"]).index] = self.avgs_.get((store, dept), 0)
                continue

            future_df = group_df[["ds"]].copy()
            forecast = model.predict(future_df)
            preds.loc[group_df.set_index([self.store_level, self.dept_level, "ds"]).index] = forecast["yhat"].values

        return preds.to_numpy()

In [16]:
categorical = ["Store", "Dept", "Type", "IsHoliday"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = added_features

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_transformer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformerWithNames(
    transformers=[
        ("num", numerical_transformer, numerical + engineered),
        ("cat", categorical_transformer, categorical),
    ],
    sparse_threshold=0.0
)

pipeline = Pipeline([
    ("index_keeper", MultiIndexKeeper()),
    ("date_features", DateFeatureCreator()),
    ("preprocessor", preprocessor),
    ("column_dropper", ColumnDropper(columns=lag_features)),
    ("prophet", ProphetRegressor())
])

In [10]:
TRAIN_ON_ENTIRE_DATA = False
print(TRAIN_ON_ENTIRE_DATA)

False


In [11]:
# Load and merge data
df_train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
df_features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
df_stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(df_stores, on="Store", how="left")
df = df.sort_values(by=['Date', 'Store', 'Dept'])

with mlflow.start_run(run_name="Feature_Engineering"):
    mlflow.log_param("features_added", added_features)

y = df["Weekly_Sales"]
X = df.drop(columns=["Weekly_Sales"])

split_idx = int(len(X) * 0.8)
X_train, X_val = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_val = y.iloc[:split_idx], y.iloc[split_idx:]

if TRAIN_ON_ENTIRE_DATA:
    X_train = X.copy()
    y_train = y.copy()

🏃 View run Feature_Engineering at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/9/runs/b29c01eab2f746248d70033a69cdd2be
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/9


In [ ]:
with mlflow.start_run(run_name="Training"):
    pipeline.fit(X_train, y_train)
    best_model = pipeline

    if not TRAIN_ON_ENTIRE_DATA:
        preds = pipeline.predict(X_val)
        mae = mean_absolute_error(y_val, preds)
        weights = X_val["IsHoliday"].apply(lambda x: 5 if x else 1)
        wmae = (weights * np.abs(y_val - preds)).sum() / weights.sum()
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("WMAE", wmae)
        # print("y:\n", y)
        # print("preds:\n", preds)

    model_path = "model.pkl"
    joblib.dump(best_model, model_path)
    mlflow.log_artifact(model_path)
    
    # mlflow.sklearn.log_model(best_model, artifact_path="model")